In [1]:
import os
import sys
import random
import warnings
import cv2
import time
import numpy as np
import pandas as pd
import time

import matplotlib.pyplot as plt
import glob
from tqdm import tqdm          ## 顯示進度條
import tensorflow as tf
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.preprocessing.image import array_to_img
from keras.models import Model, load_model
from keras.layers import Input, BatchNormalization
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
## 使用GPU
from keras import backend as K
K.tensorflow_backend._get_available_gpus()


Using TensorFlow backend.


['/job:localhost/replica:0/task:0/device:GPU:0']

In [ ]:
# import jtplot submodule from jupyterthemes
from jupyterthemes import jtplot

# currently installed theme will be used to
# set plot style if no arguments provided
jtplot.style()

In [ ]:
## 因為 plt 轉為 RGB 要使用 CV 寫入圖片要再轉回 BGR
def cv_write(fileName, img):
    r,g,b = cv2.split(img)
    img = cv2.merge([b,g,r])
    cv2.imwrite(fileName, img)

##   將原圖與預測出來的遮罩合併在一起，去掉背景
##   imgPath : 圖片路徑
##   model : 訓練完的模型
def removeBackground(imgPath, model):
    path = imgPath.split('/')[-1]
    imgOrg = cv2.imread(imgPath)
    width, height = imgOrg.shape[:2]
    img = cv2.resize(imgOrg, (256, 256))
    img = img.astype(np.float32) / 255.0
    img = np.expand_dims(img, axis=0)
    mask = model.predict(img)
    mask = (mask > 0.5).astype(np.uint8)                    ### 因為 sigmoid 結果為 0~1 之間，所以要取一半作為臨界值
    mask = np.squeeze(mask)                                 #### 將 mask 降維   (256,256,1) -> (256,256)
    mask = cv2.resize(mask, (height, width), interpolation=cv2.INTER_CUBIC)
    b,g,r = cv2.split(imgOrg)       # get b,g,r
    imgOrg = cv2.merge([r,g,b])     # switch it to rgb
    remove = cv2.bitwise_and(imgOrg, imgOrg, mask= mask)
    cv_write('./humanparsing/ModifyPredict/%s' % path, remove)
    plt.figure(figsize=(15,15))
    plt.subplot(121)
    plt.grid(False)
    plt.imshow(imgOrg)
    plt.subplot(122)
    plt.grid(False)
    plt.imshow(remove)


In [3]:
# Define IoU metric
def mean_iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred_ = tf.to_int32(y_pred > t)
        score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
    return K.mean(K.stack(prec), axis=0)

In [4]:
model = load_model('./humanparsing/Weight/model-person_seg.h5', custom_objects={'mean_iou': mean_iou})

ResourceExhaustedError: OOM when allocating tensor with shape[3,3,1024,1024] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: conv2d_10/truncated_normal/TruncatedNormal = TruncatedNormal[T=DT_INT32, dtype=DT_FLOAT, seed=87654321, seed2=4690714, _device="/job:localhost/replica:0/task:0/device:GPU:0"](conv2d_10/truncated_normal/shape)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op u'conv2d_10/truncated_normal/TruncatedNormal', defined at:
  File "/home/user/anaconda3/envs/tensorflow/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/user/anaconda3/envs/tensorflow/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/user/anaconda3/envs/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    ap

In [ ]:
start = time.time()
removeBackground('./test_data/images/2.jpeg', model)
end = time.time()
print("Programing time: %.2f" % (end - start))

In [ ]:
real = cv2.imread('./test_data/gt/gt6.PNG')
pred = cv2.imread('./humanparsing/ModifyPredict/6.jpeg')
real = cv2.resize(real, (pred.shape[1], pred.shape[0]), 
                               interpolation=cv2.INTER_CUBIC)
plt.figure(figsize=(15,15))
plt.subplot(121)
plt.grid(False)
plt.imshow(real)
plt.subplot(122)
plt.grid(False)
plt.imshow(pred)

In [ ]:
def Precision_Recall(realImg, predImg, height, width, channel):
    TP = 0.0
    FP = 0.0
    FN = 0.0
    TN = 0.0
    for k in range(channel):
        for i in range(h):
            for j in range(w):
                if realImg[i, j, k] == True:
                    if predImg[i, j, k] == True:
                        TP += 1
                    elif predImg[i, j, k] == False:
                        FN += 1
                elif realImg[i, j, k] == False:
                    if predImg[i, j, k] == True:
                        FP += 1
                    elif predImg[i, j, k] == False:
                        TN += 1
    print(TP, FP, FN, TN)
    print("Precision: %.4f" % (TP / (TP + FP)))
    print("Recall: %.4f" % (TP / (TP + FN)))
    print("F1: %.4f" % ((2 * TP) / (2*TP + FP + FN)))
    return (TP / (TP + FP)), (TP / (TP + FN)), ((2 * TP) / (2*TP + FP + FN))
# Precision_Recall(real_, pred_, h, w, c)

In [ ]:
def MAE(realImg, predImg, height, width, channel):
    mae = 0.0
    for k in range(channel):
        for i in range(height):
            for j in range(width):
                mae += abs(float(predImg[i, j, k]) - float(realImg[i, j, k]))
    print("MAE : %.4f" % (mae / (height * width)))
    return (mae / (height * width))
# MAE(real, pred, h, w, c)

In [ ]:
def removeBackground(imgPath, model):
    path = imgPath.split('/')[-1]
    path = path.split('.')[0]
    imgOrg = cv2.imread(imgPath)
    width, height = imgOrg.shape[:2]
    img = cv2.resize(imgOrg, (256, 256))
    img = img.astype(np.float32) / 255.0
    img = np.expand_dims(img, axis=0)
    start = time.time()
    mask = model.predict(img)
    end = time.time()
    mask = (mask > 0.5).astype(np.uint8)                    ### 因為 sigmoid 結果為 0~1 之間，所以要取一半作為臨界值
    mask = np.squeeze(mask)                                 #### 將 mask 降維   (256,256,1) -> (256,256)
    mask = cv2.resize(mask, (height, width), interpolation=cv2.INTER_CUBIC)
    b,g,r = cv2.split(imgOrg)       # get b,g,r
    imgOrg = cv2.merge([r,g,b])     # switch it to rgb
    remove = cv2.bitwise_and(imgOrg, imgOrg, mask= mask)
#     cv_write('./humanparsing/ModifyPredict/%s.png' % path, remove)
    return (end - start)

In [7]:
precision = []
recall = []
f1 = []
mae = []
time_ = []
for i in range(23):
    i = i + 1
    print(i)
    tt = removeBackground('./test_data/images/%d.jpeg' % i, model)
    real = cv2.imread('./test_data/gt/gt%d.PNG' % i)
    pred = cv2.imread('./humanparsing//%d.png' % i)
    real = cv2.resize(real, (pred.shape[1], pred.shape[0]), 
                                   interpolation=cv2.INTER_CUBIC)
    real_ = real.astype(np.bool)
    pred_ = pred.astype(np.bool)
    h = pred.shape[0]
    w = pred.shape[1]
    c = pred.shape[2]
    p, r, f = Precision_Recall(real_, pred_, h, w, c)
    ma = MAE(real, pred, h, w, c)
    precision.append(p)
    recall.append(r)
    f1.append(f)
    mae.append(ma)
    time_.append(tt)
    print("===============================================")
precision = np.array(precision).reshape(-1,1)
recall = np.array(recall).reshape(-1,1)
f1 = np.array(f1).reshape(-1,1)
mae = np.array(mae).reshape(-1,1)
time_ = np.array(time_).reshape(-1,1)
saveExcel = np.hstack((precision, recall, f1, mae, time_))   

1
(721602.0, 24636.0, 9468.0, 1088574.0)
Precision: 0.9670
Recall: 0.9870
F1: 0.9769
MAE : 5.3618
2
(207642.0, 2658.0, 14978.0, 1597186.0)
Precision: 0.9874
Recall: 0.9327
F1: 0.9593
MAE : 4.9316
3
(980957.0, 503.0, 24191.0, 914349.0)
Precision: 0.9995
Recall: 0.9759
F1: 0.9876
MAE : 5.2795
4
(740094.0, 526.0, 18031.0, 1083577.0)
Precision: 0.9993
Recall: 0.9762
F1: 0.9876
MAE : 5.1962
5
(883782.0, 15869.0, 35445.0, 878938.0)
Precision: 0.9824
Recall: 0.9614
F1: 0.9718
MAE : 7.2120
6
(8938795.0, 51334.0, 632485.0, 10870188.0)
Precision: 0.9943
Recall: 0.9339
F1: 0.9632
MAE : 12.5361
7
(688919.0, 7711.0, 32171.0, 968275.0)
Precision: 0.9889
Recall: 0.9554
F1: 0.9719
MAE : 4.9229
8
(922158.0, 54125.0, 29348.0, 803171.0)
Precision: 0.9446
Recall: 0.9692
F1: 0.9567
MAE : 10.2054
9
(381423.0, 375.0, 36736.0, 1390268.0)
Precision: 0.9990
Recall: 0.9121
F1: 0.9536
MAE : 7.4794
10
(9731752.0, 157880.0, 178508.0, 10424662.0)
Precision: 0.9840
Recall: 0.9820
F1: 0.9830
MAE : 4.9646
11
(356933.0,

In [8]:
excel = pd.DataFrame(saveExcel)
excel.columns = ['Precision', 'Recall', 'F1-Measure', 'MAE', 'time']
excel.to_csv('./modelModify_Precision_Recall.csv',float_format='%.4f')

## 風景與人物合成

In [ ]:
import cv2
 
# Read the images
foreground = cv2.imread("./DataAndMask/Original/man-person-jumping-desert.png")
background = cv2.imread("./DataAndMask/g3HmWc0.jpg")
alpha = cv2.imread("./DataAndMask/label/man-person-jumping-desert.png")
background = cv2.resize(background, (foreground.shape[1], foreground.shape[0]), 
                                interpolation=cv2.INTER_CUBIC)
 
# Convert uint8 to float
foreground = foreground.astype(float)
background = background.astype(float)
 
# Normalize the alpha mask to keep intensity between 0 and 1
alpha = alpha.astype(float)/255
 
# Multiply the foreground with the alpha matte
foreground = cv2.multiply(alpha, foreground)
 
# Multiply the background with ( 1 - alpha )
background = cv2.multiply(1.0 - alpha, background)
 
# Add the masked foreground and background.
outImage = cv2.add(foreground, background)

outImage = outImage/255
saveImg = np.zeros((foreground.shape[0], foreground.shape[1], 3))
cv2.normalize(outImage, saveImg, 0, 255, cv2.NORM_MINMAX)
print(outImage.shape)

# Display image
plt.imshow(outImage)
cv2.imwrite('tt.png', saveImg)
# cv2.imshow("outImg", outImage/255)
# cv2.waitKey(0)

## 將黑色背景變透明

In [ ]:
for i in range(23):
    i = i+1
    image = cv2.imread('./humanparsing/OriginalPredict/%d.png' % i)
#     image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2BGRA)
    image[np.all(image == [0, 0, 0, 255], axis=2)] = [0, 0, 0, 0]      #### 將 alpha 設為 0 
    cv2.imwrite('./humanparsing/OriginalPredict/%d.png' % i, image)